# Voorbeeld van omzetting door Veg2Hab

Dit notebook doorloopt alle stappen die tot nu toe geimplementeerd zijn voor Veg2Hab. Het is een voorbeeld van hoe de omzetting van vegetatie naar habitattypekaart kan verlopen.

Eerst worden alle benodigde resources geinstantieerd (was-wordt lijst, definitietabel, fgr-kaart). Hierna wordt de omzetting stap voor stap uitgevoerd.

In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_waswordtlijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.io.cli import CLIInterface
import pandas as pd
from veg2hab.bronnen import FGR, Bodemkaart, LBK
from veg2hab.enums import WelkeTypologie

pd.set_option('display.max_columns', 100)

CLIInterface.get_instance()

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_waswordtlijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(3)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,rVvN,VvN,SBB
0,None,14rg10,14-l
1,None,None,7-a
2,None,None,8b-b


### Definitietabel opschonen en inladen

VvN- en SBB-codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [3]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.json")
path_in_mozaiekjson = Path("../data/mozaiekjson.json")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head()

,DT regel,Habitattype,Kwaliteit,SBB,VvN,mits,mozaiek,mitsjson,mozaiekjson,Criteria,Mozaiekregel
0,2,H1110_A,Kwaliteit.GOED,50a,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
1,3,H1110_A,Kwaliteit.GOED,50b,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
2,4,H1110_A,Kwaliteit.GOED,50c,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
3,5,H1110_B,Kwaliteit.GOED,50a,None,mits in de grofzandige delen van FGR Getijdeng...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
4,6,H1110_B,Kwaliteit.GOED,50b,None,mits in de grofzandige delen van FGR Getijdeng...,<NA>,"{""type"": ""EnCriteria"", ""sub_criteria"": [{""type...","{""type"": ""GeenMozaiekregel""}",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)


## Inladen van een kartering vanaf een shapefile

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 250)
pd.set_option('display.width', 300)

In [5]:
shp_path = Path("../testing/vegetatiekarteringen/DR/BG_meerdere_terreinen2020/BG_2020_Arnoud.shp")
kartering = Kartering.from_shapefile(shp_path, vegtype_col_format="single", welke_typologie=WelkeTypologie.rVvN, ElmID_col="OBJECTID", SBB_col=[], VvN_col=[], rVvN_col=["rVVN_Code"])
kartering.gdf

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,ElmID,Area,Datum,Opm,VegTypeInfo,_LokVegTyp,_LokVrtNar,geometry
0,104,1454.893136,None,None,"[(100.0%, SBB: [], VvN: [], rVvN: ['r35aa1'])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r35aa1,"POLYGON ((241590.744 522102.389, 241545.247 522101.115, 241515.093 522101.282, 241502.201 522101..."
1,105,952.993644,None,None,"[(100.0%, SBB: [], VvN: [], rVvN: ['r35aa1'])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r35aa1,"POLYGON ((241302.776 522108.573, 241300.878 522108.203, 241300.885 522108.936, 241300.959 522116..."
2,106,7781.323761,None,None,"[(100.0%, SBB: [], VvN: [], rVvN: ['r43rg2'])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r43rg2,"POLYGON ((241549.477 522150.254, 241301.202 522141.990, 241301.384 522160.887, 241301.495 522172..."
3,107,1560.373926,None,None,"[(100.0%, SBB: [], VvN: [])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN.,"POLYGON ((241446.447 522187.268, 241301.578 522181.252, 241301.652 522189.031, 241301.703 522194..."
4,108,935.753765,None,None,"[(100.0%, SBB: [], VvN: [])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN.,"POLYGON ((241457.495 522187.726, 241457.728 522197.836, 241476.817 522198.561, 241550.453 522201..."
...,...,...,...,...,...,...,...,...
57,178,7166.291605,None,None,"[(100.0%, SBB: [], VvN: [], rVvN: ['r20aa1'])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r20aa1,"MULTIPOLYGON (((224151.137 569751.841, 224156.217 569751.417, 224163.413 569752.264, 224167.435 ..."
58,179,234.006742,None,None,"[(100.0%, SBB: [], VvN: [], rVvN: ['r20aa1'])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r20aa1,"POLYGON ((224221.178 569759.575, 224221.198 569759.567, 224221.304 569759.567, 224221.178 569759..."
59,180,7518.370063,None,None,"[(100.0%, SBB: [], VvN: [], rVvN: ['r20aa1'])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r20aa1,"POLYGON ((224181.499 569516.209, 224165.001 569534.712, 224155.476 569552.493, 224116.529 569600..."
60,187,3168.542378,None,None,"[(100.0%, SBB: [], VvN: [])]",Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN.,"POLYGON ((224217.076 569603.439, 224221.451 569596.876, 224226.102 569588.238, 224230.752 569576..."


## Inladen en omzetten van een access kartering

- Voor ieder vlak in de kartering wordt de (complexe) SBB code opgezocht.
- Deze SBB-codes worden opgeschoond en gevalideerd.
- De SBB codes worden toegevoegd aan de dataframe van de kartering als VegTypeInfo instance.
  
De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [6]:
# shape_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
# csvs_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013.mdb")
# shape_elm_id_column = "ElmID"

# access_kartering = Kartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

# access_kartering.gdf.head(3)


### Inladen bronnen

In [7]:
mask = kartering.get_geometry_mask()
fgr = FGR(Path("../data/bronbestanden/FGR.json"))
bodemkaart = Bodemkaart.from_file(Path("../data/bronbestanden/bodemkaart.gpkg"), mask=mask)
lbk = LBK.from_file(Path("../data/bronbestanden/lbk.shp"), mask=mask)

#### Opzoeken en toevoegen VvN aan kartering

In [8]:
kartering.apply_wwl(wwl)
# Voorbeeld van 3 vegtypeinfos in 1 geometry
kartering.gdf["VegTypeInfo"]

0                                      [(100.0%, SBB: ['34a1'], VvN: ['34aa1'])]
1                                      [(100.0%, SBB: ['34a1'], VvN: ['34aa1'])]
2                                     [(100.0%, SBB: ['40a-b'], VvN: ['40rg2'])]
3                                                   [(100.0%, SBB: [], VvN: [])]
4                                                   [(100.0%, SBB: [], VvN: [])]
                                         ...                                    
57                                    [(100.0%, SBB: ['20a1a'], VvN: ['20aa1'])]
58                                    [(100.0%, SBB: ['20a1a'], VvN: ['20aa1'])]
59                                    [(100.0%, SBB: ['20a1a'], VvN: ['20aa1'])]
60                                                  [(100.0%, SBB: [], VvN: [])]
61    [(100.0%, SBB: ['10-l', '9-k', '11-l'], VvN: ['10dg1', '10dg1', '10dg1'])]
Name: VegTypeInfo, Length: 62, dtype: object

#### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [9]:
kartering.apply_deftabel(dt)
kartering.gdf["HabitatVoorstel"]

0     [[onderbouwend_vegtype=VvN(klasse='34', orde='a', verbond='a', associatie='1', subassociatie=Non...
1     [[onderbouwend_vegtype=VvN(klasse='34', orde='a', verbond='a', associatie='1', subassociatie=Non...
2     [[onderbouwend_vegtype=VvN(klasse='40', orde=None, verbond=None, associatie=None, subassociatie=...
3     [[onderbouwend_vegtype=None vegtype_in_dt=None habtype='H0000' kwaliteit=<Kwaliteit.NVT: 'Nvt'> ...
4     [[onderbouwend_vegtype=None vegtype_in_dt=None habtype='H0000' kwaliteit=<Kwaliteit.NVT: 'Nvt'> ...
                                                     ...                                                 
57    [[onderbouwend_vegtype=VvN(klasse='20', orde='a', verbond='a', associatie='1', subassociatie=Non...
58    [[onderbouwend_vegtype=VvN(klasse='20', orde='a', verbond='a', associatie='1', subassociatie=Non...
59    [[onderbouwend_vegtype=VvN(klasse='20', orde='a', verbond='a', associatie='1', subassociatie=Non...
60    [[onderbouwend_vegtype=None vegtype_in_d

In [10]:
# Voorbeeld met voor 2 VegTypeInfo's waarbij de eerste twee mogelijke voorstellen heeft
kartering.gdf["HabitatVoorstel"][45]

[[HabitatVoorstel(onderbouwend_vegtype=None, vegtype_in_dt=None, habtype='H0000', kwaliteit=<Kwaliteit.NVT: 'Nvt'>, idx_in_dt=None, mits=GeenCriterium(cached_evaluation=None), mozaiek=GeenMozaiekregel(mozaiek_threshold=95, mozaiek_als_rand_threshold=50, cached_evaluation=<MaybeBoolean.TRUE: 5>), match_level=<MatchLevel.NO_MATCH: 0>, mozaiek_dict=None)]]

### De mitsen en mozaiekregels checken en de HabitatVoorstellen omzetten naar HabitatKeuzes

In [11]:
# access_kartering.bepaal_habitatkeuzes(fgr, bodemkaart, lbk)
kartering.bepaal_mits_habitatkeuzes(fgr, bodemkaart, lbk)
kartering.bepaal_mozaiek_habitatkeuzes()

### Functionele samenhang

In [12]:
kartering.functionele_samenhang()

### De kartering formatten als een Habitattypenkartering

Dit pakt de HabitatVoorstellen uit en format het geheel als een Habitattypenkartering zoals beschreven in Gegevens Leverings Protocol (uitvraag bijlage 3a).

Voor nu checken we nog geen mitsen en mozaiek en wordt voor ieder VegTypeInfo het eerste HabitatVoorstel gebruikt.

In [13]:
final_format = kartering.as_final_format()
final_format

,Area,Opm,Datum,ElmID,geometry,f_Samnvttng,f_LokVegTyp,f_LokVrtNar,Habtype1,Perc1,Opp1,Kwal1,SBB1,VvN1,f_SBBdftbl1,f_VvNdftbl1,f_Mits_info1,f_V2H_bronnen_info1,f_Mozk_info1,f_MozkPerc1,f_Status1,f_Uitleg1
0,1454.893136,None,None,104,"POLYGON ((241590.744 522102.389, 241545.247 522101.115, 241515.093 522101.282, 241502.201 522101...",100.0% H0000,Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r35aa1,H0000,100,1454.89,X,34a1,34aa1,None,None,,,,,KeuzeStatus.VEGTYPEN_NIET_IN_DEFTABEL,De vegetatietypen van het vlak zijn op geen enkel syntaxonomisch niveau in de definitietabel gev...
1,952.993644,None,None,105,"POLYGON ((241302.776 522108.573, 241300.878 522108.203, 241300.885 522108.936, 241300.959 522116...",100.0% H0000,Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r35aa1,H0000,100,952.994,X,34a1,34aa1,None,None,,,,,KeuzeStatus.VEGTYPEN_NIET_IN_DEFTABEL,De vegetatietypen van het vlak zijn op geen enkel syntaxonomisch niveau in de definitietabel gev...
2,7781.323761,None,None,106,"POLYGON ((241549.477 522150.254, 241301.202 522141.990, 241301.384 522160.887, 241301.495 522172...",100.0% HXXXX,Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r43rg2,HXXXX,100.0,7781.323761366259,X,40a-b,40rg2,---\n---\n---,"['40rg2', 468, 'H7110_A']\n['40rg2', 534, 'H7120']\n['40rg2', 652, 'H91D0']","[40rg2, H7110_A, ((Niet geautomatiseerd: het Hoogveenmos-verbond aanwezig is en een acrotelm aan...",Dit is mogelijk Herstellend hoogveen want LBK code nan.\nDit is mogelijk toch Hoogveenlandschap ...,"[40rg2, H7110_A, Geen mozaiekregel (altijd waar), TRUE]\n[40rg2, H7120, Geen mozaiekregel (altij...",,KeuzeStatus.NIET_GEAUTOMATISEERD_CRITERIUM,Er zijn niet-geautomatiseerde mitsen/mozaiekregels gevonden; deze kunnen niet door veg2hab worde...
3,1560.373926,None,None,107,"POLYGON ((241446.447 522187.268, 241301.578 522181.252, 241301.652 522189.031, 241301.703 522194...",100.0% H0000,Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN.,H0000,100,1560.37,X,,,None,None,,,,,KeuzeStatus.GEEN_OPGEGEVEN_VEGTYPEN,Er zijn in de vegetatiekartering geen vegetatietypen opgegeven voor dit vlak. veg2hab kent aan d...
4,935.753765,None,None,108,"POLYGON ((241457.495 522187.726, 241457.728 522197.836, 241476.817 522198.561, 241550.453 522201...",100.0% H0000,Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN.,H0000,100,935.754,X,,,None,None,,,,,KeuzeStatus.GEEN_OPGEGEVEN_VEGTYPEN,Er zijn in de vegetatiekartering geen vegetatietypen opgegeven voor dit vlak. veg2hab kent aan d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,7166.291605,None,None,178,"MULTIPOLYGON (((224151.137 569751.841, 224156.217 569751.417, 224163.413 569752.264, 224167.435 ...",100.0% HXXXX,Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r20aa1,HXXXX,100,7166.29,X,20a1a,20aa1,None,"['20aa1', 393, 'H4030']","[20aa1, H4030, (niet (FGR is Noordzee (F) of FGR is Getijdengebied (F) of FGR is Duinen (F) of (...","FGR type is niet Noordzee, maar Hogere Zandgronden.\nFGR type is niet Getijdengebied, maar Hoger...","[20aa1, H4030, Geen mozaiekregel (altijd waar), TRUE]",,KeuzeStatus.NIET_GEAUTOMATISEERD_CRITERIUM,Er zijn niet-geautomatiseerde mitsen/mozaiekregels gevonden; deze kunnen niet door veg2hab worde...
58,234.006742,None,None,179,"POLYGON ((224221.178 569759.575, 224221.198 569759.567, 224221.304 569759.567, 224221.178 569759...",100.0% HXXXX,Geen kolommen opgegeven voor lokale vegetatietypen,Lokale typologie is primair vertaald naar rVvN. r20aa1,HXXXX,100,234.007,X,20a1a,20aa1,None,"['20aa1', 393, 'H4030']","[20aa1, H4030, (niet (FGR is Noordzee (F) of FGR is Getijdengebied (F) of FGR is Duinen (F) of (...","FGR type is niet Noordzee, maar Hogere Zandgrond

In [14]:
kartering.final_format_to_file(Path("../testing/veg2hab_output_rvvn_test.shp"))

/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1647: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final.to_file(path)
[40rg2, H7120, LBK is Herstellend hoogveen (C), CANNOT_BE_AUTOMATED]
[40rg2, H91D0, (Niet geautomatiseerd: mits buiten H7110_A en H7120), CANNOT_BE_AUTOMATED]' of field f_Mits_inf has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
